# Prepare Haute-Garonne Sirene registry

## Imports

In [4]:
%run "../config/notebook.ipynb"
%run "../config/files.ipynb"
%run "../utils/stop_execution.ipynb"

# import communes of the Haute-Garonne
%run "../prepare/prepare_townships_hautegaronne.ipynb"

import pandas as pd
import numpy as np

df_hautegaronne_townships: 578 townships entries


## Read from cache

In [5]:
if 'FROM_CACHE' in locals() or 'FROM_CACHE' in globals():
    if FROM_CACHE:
        df_sirenes_hautegaronne = pd.read_csv(PREPARED_SIRENES_HAUTEGARONNE_FILE_PATH, index_col=['siret'])
        print("(from cache) df_sirenes_hautegaronne: {} companies".format(len(df_sirenes_hautegaronne.index)))        
        raise StopExecution

## Create the sirene dataframe

 ### Read the sirene csv file

In [64]:
# keep only columns
#  0   siren                          int64  
#  2   siret                          int64  
#  4   dateCreationEtablissement      object 
#  16   trancheEffectifsEtablissement  object 
#  20   codePostalEtablissement        float64
#  5   codeCommuneEtablissement        object
df_sirenes = pd.read_csv(COMPANIES_STATISTICS_FILE_PATH, usecols=[0,2,4,5,20], \
                         dtype={'siren': np.int64, \
                               'siret': np.int64, \
                               'dateCreationEtablissement': np.str, \
                                'trancheEffectifsEtablissement': np.str, \
                               'codeCommuneEtablissement': np.str}, na_values=['NaN', 'NN'])

In [65]:
df_sirenes['trancheEffectifsEtablissement'].fillna('01', inplace=True)

### Rename columns

In [70]:
df_sirenes.columns = ['siren', 'siret', 'creation_date', 'staffing_range', 'code_insee']

### Add a creation_year column

In [71]:
df_sirenes['year'] = df_sirenes['creation_date'].apply(lambda d: str(d).split('-')[0])

### Filter on Haute-Garonne communes

In [72]:
df_sirenes['department'] = df_sirenes['code_insee'].apply(lambda x: str(x)[:-3])

In [73]:
df_sirenes_hautegaronne = df_sirenes[df_sirenes['department'] == '31']

### Check on siret

In [79]:
sr_siret_groupby_count = df_sirenes_hautegaronne.groupby('siret')['siret'].count()

In [88]:
if len(sr_siret_groupby_count[sr_siret_groupby_count > 1]) > 0:
    print("WARNING: some rows have the same SIRET identifier.")

### Set the index

In [89]:
df_sirenes_hautegaronne = df_sirenes_hautegaronne.set_index(['siret'])

### Save the dataframe

In [91]:
df_sirenes_hautegaronne.to_csv(PREPARED_SIRENES_HAUTEGARONNE_FILE_PATH)
print("df_sirenes_hautegaronne: {} companies".format(len(df_sirenes_hautegaronne.index)))

df_sirenes: 598978 companies
